In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Activation, Input #changed 'keras.' to 'tensorflow.keras.'
from tensorflow.keras.models import Sequential, Model #changed 'keras.' to 'tensorflow.keras.'
from tensorflow.keras import optimizers #changed 'keras.' to 'tensorflow.keras.'
import cv2
import math
import glob
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential
from IPython.display import display
from keras_vggface.vggface import VGGFace
import sys

In [2]:
model = VGGFace()
model.summary()

In [3]:
model.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

In [4]:
temp = Sequential()
for layer in model.layers[0:-4]: # just exclude last two layers from copying
    temp.add(layer)

for layer in temp.layers[0:9]:
    layer.trainable = False
temp.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 56, 56, 256)       2

In [5]:
temp.save_weights('tempweights.h5')

In [ ]:
BATCH_SIZE = 16
IMG_SIZE = (224, 224)

directory = "newtrain"
train_ds = image_dataset_from_directory(
    directory, label_mode=None, color_mode='rgb', batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=True, seed=0, validation_split=None, subset=None,
    interpolation='bicubic', follow_links=False)
directory = "newval"
valid_ds = image_dataset_from_directory(
    directory, label_mode=None, color_mode='rgb', batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=True, seed=0, validation_split=None, subset=None,
    interpolation='bicubic', follow_links=False)

In [4]:
SR = keras.models.load_model('Models/SR/SRnet')
SR.load_weights('Models/SR/epoch101-110/SRnetweights.h5')

In [5]:
'''callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=7,
                              verbose=0, mode='min', restore_best_weights=False)'''

input_img = Input(shape=(224,224,3))
A5 = SR(input_img) 
A6 = temp(A5)
SRFECNN = Model(input_img, A6)
opt = optimizers.Adam(learning_rate = 0.000003)
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

SRFECNN.compile(
    optimizer=opt,
    loss=root_mean_squared_error,
    metrics=['accuracy'])
for layer in SRFECNN.layers[0:-1]:
    layer.trainable = False
#SRFECNN.load_weights('Models/twobranch/twobranch_weights_e1_rms.h5')
#SRFECNN.compile(optimizer=opt,loss='root_mean_squared_error',metrics=['accuracy'])
SRFECNN.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 224, 224, 3)       36723     
_________________________________________________________________
sequential (Sequential)      (None, 4096)              117479232 
Total params: 117,515,955
Trainable params: 115,743,744
Non-trainable params: 1,772,211
_________________________________________________________________


ValueError: axes don't match array

In [8]:
FECNN = Sequential()
for layer in model.layers[0:-4]: # just exclude last two layers from copying
    FECNN.add(layer)
for layer in FECNN.layers:
    layer.trainable = False


#FECNN.save('Models/FECNN')
#SRFECNN.summary()

In [9]:
# to generate training images, we first need to downscale the image by scaler factor and then upscale it back to the original size
def create_LR(image): 
    image = image / 255 #normalising the pixel values
    image = tf.image.resize(image, [24, 24], method="bicubic") #for 36x35, batches~75, loss~4000, accuracy~62
    image = tf.image.resize(image, [224, 224], method="bicubic")
    return image

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(
    lambda x: (create_LR(x), FECNN(x))
)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x: (create_LR(x), FECNN(x))
)
valid_ds = valid_ds.prefetch(buffer_size=AUTOTUNE)

In [22]:
SRFECNN.load_weights('Models/twobranch_overfit/5unfreezed_epoch30.h5')
#SRFECNN.load_weights('Models/twobranch_overfit/5unfreezed_epoch30.h5')

input_img = Input(shape=(224,224,3))
A5 = SRFECNN.layers[2](input_img) 
#A6 = SRFECNN.layers[2](A5)
net = Model(input_img, A5)
#net.add(keras.Input(shape=(224,224,3)))
#for layer in SRFECNN.layers[2:]: # just exclude last two layers from copying
    #net.add(layer)
net.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 4096)              117479232 
Total params: 117,479,232
Trainable params: 0
Non-trainable params: 117,479,232
_________________________________________________________________


In [ ]:
import time
start = time.time()
history = SRFECNN.fit(train_ds, epochs=1, validation_data=valid_ds)
end = time.time()
print(end - start)

In [ ]:
#SRFECNN.save('Models/SRFECNN')
#SRFECNN.save('Models/twobranch/twobranch_weights_e1.h5')

In [ ]:
SRFECNN.save('Models/twobranch/twobranch_weights_e1_rms.h5')

In [25]:
dic = dict()
i=0
TP = 0
for filename in os.listdir('SRtrain/train'):
    img = cv2.imread(os.path.join('SRtrain/train',filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    imgr = np.array([img])
    imgr = tf.convert_to_tensor(imgr)
    vec = FECNN.predict(imgr)
    dic['s'+ str(i)] = vec
    i = i+1
i=0
FP=0
TN=0
FN=0
accuracy = 0
acc = 0
a = np.zeros((50,50))
threshold = 0.235
for filename in os.listdir('finaltest/test'):
    img = cv2.imread(os.path.join('finaltest/test',filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (24,24), interpolation=cv2.INTER_CUBIC)
    img1 = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    imgr = np.array([img1])
    imgr = tf.convert_to_tensor(imgr)
    vec1 = SRFECNN.predict(imgr)
    vec1 = vec1/np.linalg.norm(vec1)
    a = []
    for j in range(100):
        k = 's' + str(j)
        vec2 = dic[k]/np.linalg.norm(dic[k])
        dist = np.linalg.norm(vec2-vec1)
        dist = dist.flatten()
        a.append(dist)
    Dist_pred.append(min(a))
    ID_pred.append(np.argmin(a))
for i in Dist_pred:
    
        if dist.numpy()<=5:
            a[i,int(j/10)] += 1

        if dist.numpy()<=5 and j==int(i/5):
            TP = TP + 1
            accuracy = accuracy + 1
        elif dist.numpy()<=5 and j!=int(i/5): 
            FP = FP + 1
        
        #print("dist = ",dist,"subject = ",j,"i = ",i)
    i = i+1

b = np.argmax(a, axis=1)
for i in range(50):
    if i==b[i]:
        acc += 1
print(a)
print(b)
print(acc*2)
P = TP/(TP+FP)
R = TP/(TP+FN)
F1 = 2*P*R/(P+R)
print(P)
print((accuracy/50)*100)


[[ 5.  0.  7. ...  0.  0.  0.]
 [ 5.  0.  0. ...  0.  0.  0.]
 [ 5. 10.  8. ... 10. 10. 10.]
 ...
 [ 5. 10. 10. ... 10. 10. 10.]
 [ 5. 10. 10. ... 10. 10. 10.]
 [ 5. 10.  3. ...  8.  8. 10.]]
[ 2  0  1  2  1  1  1  1  0  1  1  1  1  3  1  1  1  1  1  1  1  1  1  1
  1  1  1  0  1  1  1  1  1  3  1  1  1  1 33  1  1  1  1  0  1  1  1  1
  1  1]
0
0.0013310327828444663
54.0


In [ ]:
dic = dict()
i=0
accuracy = 0
threshold = 2.2
for filename in os.listdir('finaltrain/train'):
    img = cv2.imread(os.path.join('finaltrain/train',filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgr = np.array([img])
    imgr = tf.convert_to_tensor(imgr)
    vec = FECNN.predict(imgr)
    dic['s'+ str(i)] = vec
    i = i+1
i=0
dic1 = dict()
for filename in os.listdir('finaltest/test'):
    img = cv2.imread(os.path.join('finaltest/test',filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img1 = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
    imgr = np.array([img1])
    imgr = tf.convert_to_tensor(imgr)
    vec1 = SRFECNN.predict(imgr)
    vec1 = vec1/np.linalg.norm(vec1)
    temp = 9999999
    for j in range(500):
        k = 's' + str(j)
        vec2 = dic[k]/np.linalg.norm(dic[k])
        dist = tf.reduce_sum(K.sqrt(K.square(vec2-vec1)))
        if(dist.numpy()<temp):
            temp = dist.numpy()
            dic1[i].append(j)
            dic1[i].append(temp)
    i = i+1
for test, subject, distance in dic1.items():
    if(int(test/5)==int(subject/10) and distance<threshold):
        accuracy = accuracy + 1
                    
print((accuracy/250)*100)

In [ ]:
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Mean Squared Error')
plt.ylim([0,100])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
img = cv2.imread('train/train/ATT_49_1988739.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#display(array_to_img(img))
#print(img)
#plt.imshow(img)
img = img/255
img1 = cv2.resize(img, (224,224), interpolation=cv2.INTER_CUBIC)
img1 = cv2.resize(img, (36,36), interpolation=cv2.INTER_CUBIC)
img1 = cv2.resize(img1, (224,224), interpolation=cv2.INTER_CUBIC)
#img1 = img1*255
plt.imshow(img1)

#cv2.imwrite('HR1.bmp', img1)
#original = cv2.imread("HR1.bmp")
#print(img.shape)

In [ ]:
imgr = np.array([img1])
imgr = tf.convert_to_tensor(imgr)
img_HR = model.predict(imgr)
#print(img_HR)
img2 = np.array(img_HR,dtype=np.uint8)
img2 = img2[0,:]
plt.imshow(img2)